# Real vs fake news prediction model

## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats import weightstats as stests
from collections import Counter
from ipynb.fs.full.bespoke_functions import plot_comparison_histogram, get_number_capitals
from ipynb.fs.full.bespoke_functions import get_most_common_words, get_avg_word_length

## Read in datasets

In [2]:
real_df = pd.read_csv('~/documents/Data/Fake vs Real News/real_eng.csv')
fake_df = pd.read_csv('~/documents/Data/Fake vs Real News/fake_eng.csv')

## Transform data to correct format for sklearn modelling

In [3]:
news_df = pd.concat([real_df, fake_df])

(39104, 6)